### Customer Segmentation - RFM Model
RFM Analysis is a data-driven customer segmentation technique that aims to cluster different customers into groups based on their transactional data , particularly based on how recently they have transacted (Recency), how frequently they typically transact (Frequency), and how much they have spent so far (Monetary Value). 

The result of this Analysis can help businesses to identify their customer's behavior and also make decisons based on these attributes.

In [197]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
warnings.filterwarnings("ignore")

In [198]:
# Import Data
data = pd.read_csv(r"C:\Users\osaso\Desktop\Project\archive\online_retail_listing.csv", sep=';', encoding= 'unicode_escape')
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,"6,95",13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,"2,1",13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,"1,25",13085.0,United Kingdom


In [199]:
data.shape

(1048575, 8)

In [200]:
data.describe()

,Quantity,Customer ID
count,1.048575e+06,811893.000000
mean,9.957525e+00,15324.712265
std,1.335187e+02,1697.033034
min,-7.421500e+04,12346.000000
25%,1.000000e+00,13971.000000
50%,3.000000e+00,15260.000000
75%,1.000000e+01,16795.000000
max,7.421500e+04,18287.000000


In [201]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1048575 non-null  object 
 1   StockCode    1048575 non-null  object 
 2   Description  1044203 non-null  object 
 3   Quantity     1048575 non-null  int64  
 4   InvoiceDate  1048575 non-null  object 
 5   Price        1048575 non-null  object 
 6   Customer ID  811893 non-null   float64
 7   Country      1048575 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 64.0+ MB


#### Preprocessing

In [202]:
# check duplicates in Customer ID
data['Customer ID'].duplicated().sum()

1042650

In [203]:
data['Customer ID'].nunique()

5924

In [204]:
data['Invoice'].nunique()

52961

In [205]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])

In [206]:
data.head(1)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-01-12 07:45:00,"6,95",13085.0,United Kingdom


In [207]:
data['Price'] = data['Price'].str.replace("," , ".")

In [208]:
data['InvoiceDate'].min()

Timestamp('2009-01-12 07:45:00')

In [209]:
data['InvoiceDate'].max()

Timestamp('2011-12-10 17:19:00')

In [210]:
# filter by positive quantity
df = data[data['Quantity']>=0] # This reduces the size of the data to 1,025,878
df.shape 

(1025878, 8)

In [211]:
df['Quantity'] = df['Quantity'].astype('float')
df['Price'] = df['Price'].astype('float')

In [212]:
# The neagtive price probably implies on discount on items bought. For the purpose of this project, I excluded negative prices
df = df[df['Price']>0] # This reduces the size to 1,023,144
df.shape

(1023144, 8)

In [213]:
df.describe()

,Quantity,Price,Customer ID
count,1.023144e+06,1.023144e+06,793309.000000
mean,1.091459e+01,4.073926e+00,15331.994757
std,9.932386e+01,5.165363e+01,1696.330415
min,1.000000e+00,1.000000e-03,12346.000000
25%,1.000000e+00,1.250000e+00,13979.000000
50%,3.000000e+00,2.100000e+00,15271.000000
75%,1.000000e+01,4.130000e+00,16801.000000
max,7.421500e+04,2.511109e+04,18287.000000


In [214]:
df['TotalSum'] = df['Quantity'] * df['Price']
df.head(1)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalSum
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12.0,2009-01-12 07:45:00,6.95,13085.0,United Kingdom,83.4


In [215]:
## I created a hypothetical date to use as a starting point to calculate metrics as if we are doing analysis on the most recent data
snapshot_date = max(df.InvoiceDate) + datetime.timedelta(days=1)
snapshot_date

Timestamp('2011-12-11 17:19:00')

#### Calculate RFM Metrics

##### For RFM analysis, We need a few details of each Customer:

- Customer ID - A uniqiue identifier to identify each cuistomer
- Recency (R) as days since last purchase: How many days ago was their last purchase? 
- Frequency (F) as the total number of transactions: How many times has the customer made a purchase? 
- Monetary (M) as total money spent: How much has this customer spent? This is gotten by summing up the amount from all transactions to get the M value.

To extract these values, we only need the following columns:

‘Customer ID’, ‘Invoice’, ‘Invoice Date’ and ‘TotalSum (which was gotten by multiplying quantity and price above)’

In [216]:
#Aggregate data on customer level
new_data = df.groupby(['Customer ID']).agg({
                                    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,  #gives the number of days between hypothetical today and day of last transaction
                                     'Invoice':lambda x : len(x.unique()),
                                     'TotalSum': 'sum'})
new_data.head()

,InvoiceDate,Invoice,TotalSum
Customer ID,,,
12346.0,327,12,77556.46
12347.0,41,7,5408.50
12348.0,77,5,2019.40
12349.0,20,4,4428.69
12350.0,312,1,334.40


In [217]:
# Rename columns
new_data = new_data.rename(columns={'InvoiceDate':'Recency',
                         'Invoice' : 'Frequency',
                         'TotalSum' : 'Monetary Value'})
new_data.head(1)

,Recency,Frequency,Monetary Value
Customer ID,,,
12346.0,327,12,77556.46


#### Building RFM Segments

In [218]:
# Recency Quartile
r_labels = range(3, 0, -1) # The lower the recency value, the more recently the customer purchased
r_quartiles = pd.qcut(new_data['Recency'], 3, labels=r_labels)
new_data = new_data.assign(R = r_quartiles.values) 

In [219]:
# Frequency and Monetary Value Quartile
f_labels= range(1,4)
m_labels = range(1, 4)
f_quartiles = pd.qcut(new_data['Frequency'], 3, labels=f_labels)
m_quartiles = pd.qcut(new_data['Monetary Value'], 3, labels=m_labels)

new_data = new_data.assign(F = f_quartiles.values)
new_data = new_data.assign(M = m_quartiles.values)

In [220]:
# change the datatype of rfm segments
new_data[['R', 'F', 'M']] = new_data[['R', 'F', 'M']].astype('int')

#### Building RFM Segment and RFM Score

In [221]:
# Sum RFM Quartile value to get RFM Score
def join_rfm(x):
    return (str(x['R']) + str(x['F']) + str(x['M']))

In [222]:
new_data['RFM_Segment'] = new_data.apply(join_rfm, axis=1)
new_data['RFM_Score'] = new_data[['R', 'F', 'M']].sum(axis=1)

In [223]:
new_data.head()

,Recency,Frequency,Monetary Value,R,F,M,RFM_Segment,RFM_Score
Customer ID,,,,,,,,
12346.0,327,12,77556.46,1,3,3,1.03.03.0,7
12347.0,41,7,5408.50,3,3,3,3.03.03.0,9
12348.0,77,5,2019.40,2,2,3,2.02.03.0,7
12349.0,20,4,4428.69,3,2,3,3.02.03.0,8
12350.0,312,1,334.40,1,1,1,1.01.01.0,3


#### Analyzing RFM Table

In [224]:
new_data.groupby('RFM_Segment').size().sort_values(ascending=False)[:10] #top 10 largest rfm segments

RFM_Segment
1.01.01.0    1079
3.03.03.0     970
2.01.01.0     478
2.03.03.0     476
2.02.02.0     376
1.01.02.0     331
3.02.02.0     323
2.01.02.0     266
1.02.02.0     261
3.01.01.0     212
dtype: int64

In [225]:
# Summary metrics by RFM score
new_data.groupby('RFM_Score').agg({'Recency' : 'mean',
                                  'Frequency': 'mean',
                                   'Monetary Value' : ['mean', 'count'] }).round(1)
                                                                  

Recency Frequency Monetary Value      
             mean      mean           mean count
RFM_Score                                       
3           514.6       1.2          228.4  1079
4           291.6       1.5          454.1   857
5           212.9       2.4          791.4   860
6           153.6       3.4         1205.3   717
7           107.1       5.4         2175.9   660
8            84.6       9.5         4135.2   717
9            19.3      19.7        11075.8   970

In [226]:
# Define rfm_level function
def rfm_level(df):
    if df['RFM_Score'] >= 10:
        return 'Top'
    elif ((df['RFM_Score'] >= 6) and (df['RFM_Score'] < 10)):
        return 'Middle'
    else:
        return 'Low'

# Create a new variable RFM_Level
new_data['RFM_Level'] = new_data.apply(rfm_level, axis=1)

# Print the header with top 5 rows
print(new_data.head())

             Recency  Frequency  Monetary Value  R  F  M RFM_Segment  \
Customer ID                                                            
12346.0          327         12        77556.46  1  3  3   1.03.03.0   
12347.0           41          7         5408.50  3  3  3   3.03.03.0   
12348.0           77          5         2019.40  2  2  3   2.02.03.0   
12349.0           20          4         4428.69  3  2  3   3.02.03.0   
12350.0          312          1          334.40  1  1  1   1.01.01.0   

             RFM_Score RFM_Level  
Customer ID                       
12346.0              7    Middle  
12347.0              9    Middle  
12348.0              7    Middle  
12349.0              8    Middle  
12350.0              3       Low  


In [227]:
# Calculate average values for each RFM_Level, and return a size of each segment 
rfm_level_agg = new_data.groupby('RFM_Level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary Value': ['mean', 'count']
}).round(1)

# Print the aggregated dataset
print(rfm_level_agg)

          Recency Frequency Monetary Value      
             mean      mean           mean count
RFM_Level                                       
Low         353.4       1.7          470.7  2796
Middle       84.9      10.4         5224.8  3064


##### 
From the RFM Levels above , we can identify high value customers, medium value customers and low value customers. The count of values in each segment shows that a greater percentage of customers represented in this dataset are low value customers, with high valued customers representing a much smaller percentage.

Note: Customers can be divided into as many groups/segments based on rules defined by the business, as different business purposes will determine the types of grouping and in turn the different strategies to adopt.